# This is an attempt to build a testing battery.

In [3]:
import sys, os
import statistics
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(sys.path)
import gillespy2
from scipy import stats
import numpy

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/Seanebum/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/home/Seanebum/.local/lib/python3.5/site-packages/IPython/extensions', '/home/Seanebum/.ipython', '/mnt/c/Users/seanm/OneDrive/Documents/Research/TauHybridSolver/GillesPy2']


The performance of this package can be significantly increased if you install Cython.


In [4]:
#Doing this in known order of good solvers. I imagine I'm going to have to nudge some stuff.
#Basic ODE Is messed up. I think I remember looking into this before and it not being super difficult.
#Linting the Optimized SSA Solver stuff, coercing into standard naming pattern. (I consider Basic SSA the standard.)
#Ask about Dalton?
#C stuff does not work with my plotting function! It uses a list and not a dictionary. Hooray.....
#Removed extraneous imports from tau_leaping basic.
#Basic_Tau_Leaping doesn't work. I know it's status is temptuous.
#Regular Tau Hangs (I threw 30 minutes at it for the MM and TCE model, which are generally instataneous)
#Final List before going to research meeting is SSA, Root, and Optimized
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_tau_leaping_solver import BasicTauLeapingSolver
from gillespy2.basic_tau_hybrid_solver import BasicTauHybridSolver

In [5]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.0001)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='A', initial_value=301)
            B = gillespy2.Species(name='B', initial_value=120)
            C = gillespy2.Species(name='C', initial_value=0)
            D = gillespy2.Species(name='D', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

## Test Importation of Models.

In [9]:
modelList = [MichaelisMenten()]

solverList = [BasicSSASolver, SSACSolver, BasicTauHybridSolver, BasicTauLeapingSolver]
timingList = []


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    for i in tqdm(range(100), desc = 'Model: {0}, Solver: {1}'.format(model.name, solver)):
        standard_results = model.run(stochkit_home="/mnt/c/Users/seanm/OneDrive/Documents/Research/StochKit")
        start = timer()
        test_results =model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if solver == SSACSolver:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])


Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   0%|          | 0/100 [00:00<?, ?it/s]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   1%|          | 1/100 [00:01<02:03,  1.25s/it]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   2%|▏         | 2/100 [00:02<02:07,  1.30s/it]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   3%|▎         | 3/100 [00:04<02:11,  1.36s/it]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   4%|▍         | 4/100 [00:05<02:05,  1.31s/it]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   5%|▌         | 5/100 [00:06<02:08,  1.35s/it]
Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>:   6%|▌         | 6/100 [00:08<02:10,  1.39s/it]

KeyboardInterrupt: 

In [ ]:
print(*exterior_stats)

In [ ]:
from gillespy2.example_models import LacOperon
model = LacOperon()
start = timer()
results = model.run(solver=BasicSSASolver)
model.plot(results)

In [ ]:
#Workaround for tqdm doing some weird dynamic scaling that causes a runtime error in JupyterLab.
#Operation seems to take to long for Jupyter Lab when evaluating LacOperon Model for any solvers when coerced into a function list. Individual Stuff works well.
for x in tqdm(range(len(solverList))):
    model = LacOperon()
    start = timer()
    model.run(solver=solverList[x])
    stop = timer()
    timingList.append([LacOperon, str(solverList[x]), stop-start])

In [ ]:
from gillespy2.example_models import MichaelisMenten
import h5py
model = MichaelisMenten()
standard_results = model.run(stochkit_home="/home/jackson/StochKit/StochKit")
print(standard_results)
# with h5py.File('MichaelisMenten.hdf5','w') as f:
for species in model.listOfSpecies.keys():
     print(len(standard_results[0][f'{species}']))